# Imports

In [ ]:
import pandas as pd
from hbt_tools import messdatenbank as mdb
from pathlib import Path
from pytz import timezone

# Functions

In [ ]:
def ts_filter_date(ts, date_from=None, date_to=None):
    ts = ts.copy()
    if (date_from is None and date_to is None):
        pass

    elif (date_to is None and date_from is not None):
        ts = ts[ts.index>=date_from]
    
    elif (date_to is not None and date_from is None):
        ts = ts[ts.index<=date_to]
    else:
        ts = ts[(ts.index>=date_from) & (ts.index<=date_to)]

    return ts
#-------------------------------------------------------------------------------

def sum_per_period(ts, periods, date_format, idx_col=0):
    res = pd.DataFrame(columns=['von', 'bis', 'summe'])
    tz = timezone('Europe/Zurich')

    res_list = []
    for period in periods:
        date_start = pd.to_datetime(period[0], format=date_format)
        date_start = tz.localize(date_start)

        date_end = pd.to_datetime(period[1], format=date_format)
        date_end = tz.localize(date_end)

        ts_period = ts[(ts.index>=date_start) & (ts.index<date_end)]
        df_temp = pd.DataFrame({'von': [period[0]], 'bis': [period[1]],
            'summe': [ts_period.iloc[:,idx_col].sum()]})
        
        res_list.append(df_temp)
        
    res = pd.concat(res_list)
    
    return res

# Parameters

In [ ]:
date_format = '%d.%m.%Y %H:%M'
tz = timezone('Europe/Zurich')

date_start = pd.to_datetime('17.06.2023 00:00', format=date_format)
date_start = tz.localize(date_start)

date_end = pd.to_datetime('31.10.2023 00:00', format=date_format)
date_end = tz.localize(date_end)

ezg_enge = 1.7 # [ha]
ezg_birchi = 3.3 # [ha]

save_dir = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.40 SABA Enge Biberist"
    r"\05 Berechnungen Grundlagen\Datenauswertung")

# Regen

In [ ]:
id = 2064
rain = mdb.get_data(id, date_start, date_end, date_format)

rain_tot = rain['wert'].sum()
print(f"Regen tot: {rain_tot:,.1f} mm")

zulauf_rain_enge = rain_tot * ezg_enge * 10
print(f"Zulauf vom Regen Enge: {zulauf_rain_enge:,.0f} m3")

zulauf_rain_birchi = rain_tot * ezg_birchi * 10
print(f"Zulauf vom Regen Birchi: {zulauf_rain_birchi:,.0f} m3")

# SABA Enge

In [ ]:
periods = [
    ('09.06.2023 12:56', '10.07.2023 09:19'),
    ('10.07.2023 09:19', '28.07.2023 16:59'),
    ('28.07.2023 16:59', '19.09.2023 15:29'),
    ('19.09.2023 15:29', '14.10.2023 07:05')
]

In [ ]:
rain_periods = sum_per_period(rain, periods, date_format=date_format)
rain_periods

## Zulauf

In [ ]:
id = 1931
q_enge = mdb.get_data(id, date_start, date_end, date_format)

# Compute volume per time-step
q_enge['delta_t'] = q_enge.index
delta_t = q_enge['delta_t'] - q_enge['delta_t'].shift()
q_enge['delta_t'] = delta_t
q_enge['volumen [l]'] = q_enge['wert'] * q_enge['delta_t'].dt.total_seconds()

In [ ]:
q_enge['delta_t'].value_counts()

In [ ]:
vol_tot = q_enge['volumen [l]'].sum() / 1e3 # l -> m3
print(f"Zufluss Enge: {vol_tot:,.0f} m3")

In [ ]:
q_enge_periods = sum_per_period(q_enge, periods, date_format=date_format, idx_col=2)
q_enge_periods.iloc[:,2] /= 1e3
q_enge_periods

# SABA Birchi

In [ ]:
periods = [
    ('15.06.2023 12:12', '29.06.2023 05:34'),
    ('29.06.2023 05:34', '10.07.2023 08:53'),
    ('10.07.2023 08:53', '23.07.2023 13:25'),
    ('23.07.2023 13:25', '04.08.2023 11:53'),
    ('04.08.2023 12:52', '20.08.2023 02:54'),
    ('20.08.2023 02:54', '27.08.2023 16:04'),
    ('27.08.2023 16:04', '15.09.2023 20:30'),
    ('15.09.2023 20:30', '26.10.2023 11:48')
]

In [ ]:
rain_periods = sum_per_period(rain, periods, date_format=date_format)
rain_periods

## Zulauf

In [ ]:
id = 1922
q_birchi = mdb.get_data(id, date_start, date_end, date_format)

# Compute volume per time-step
q_birchi['delta_t'] = q_birchi.index
delta_t = q_birchi['delta_t'] - q_birchi['delta_t'].shift()
q_birchi['delta_t'] = delta_t
q_birchi['volumen [l]'] = q_birchi['wert'] * q_birchi['delta_t'].dt.total_seconds()

In [ ]:
q_birchi.to_csv((save_dir / 'Zulaufvolumen_Birchi.csv'), sep=';')
q_birchi['delta_t'].value_counts()

In [ ]:
vol_tot = q_birchi['volumen [l]'].sum() / 1e3 # l -> m3
print(f"Zufluss Birchi: {vol_tot:,.0f} m3")

In [ ]:
q_birchi_periods = sum_per_period(q_birchi, periods, date_format=date_format, idx_col=2)
q_birchi_periods.iloc[:,2] /= 1e3
q_birchi_periods